In [1]:
import os
import requests
import re
import sys
import regex as re

In [2]:
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# Graphics in SVG format are more sharp and legible
%config InlineBackend.figure_format = 'svg'
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Imports the Google Cloud client library
from google.cloud import storage
# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "nlp_final_project_kshitijm"

# Creates the new bucket
bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_final_project_kshitijm connected.


In [4]:
%%time
df_filt_2020=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data5_2021_Topics_Sentiments_NER.csv',lineterminator='\n')
df_filt_2021=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2021v2_Topics_Sentiments_NER.csv',lineterminator='\n')
df_filt_2022=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2022_Topics_Sentiments_NER.csv', lineterminator='\n')
df_filt_2023=pd.read_csv('gs://nlp_final_project_kshitijm/00_Data/NLP_FP_Data6_2023_Topics_Sentiments_NER.csv', lineterminator='\n')

CPU times: user 1min 54s, sys: 14.2 s, total: 2min 8s
Wall time: 5min 27s


In [5]:
df_filt_all=pd.concat([df_filt_2020, df_filt_2021, df_filt_2022, df_filt_2023])

In [6]:
df_filt_pos=df_filt_all[df_filt_all.sent_label==2]


In [7]:
df_filt_pos.head()

,Unnamed: 0.1,Unnamed: 0,url,date,language,title,text,cleaned_text,article_source,clean_title,...,rake_phrases_articles,rake_phrases_joined,final_topic,sent_probs,sent_label,entities_spacy,ENT_ORG,ENT_PROD,ENT_PER,ENT_NORP
1,1.0,1,https://health.economictimes.indiatimes.com/ne...,2020-01-10,en,artificial intelligence: Researchers develop A...,\n\nartificial intelligence: Researchers devel...,artificial intelligence: Researchers develop ...,NaN,artificial intelligence: Researchers develop A...,...,['economic times ethealthworldhome news hospit...,economic times ethealthworldhome news hospital...,1,[[1.9508268e-04 8.0524624e-04 9.9899966e-01]],2,"[('AI', 'PRODUCT'), ('Health News', 'ORG'), ('...","['Health News', 'ETHealthworld', 'The Economic...","['AI', 'NewsJavascript', 'AI', 'AIsystem', 'AI...","['worldleading uropathologists', 'Martin Eklun...","['Swedish', 'Chinese']"
5,5.0,5,https://www.connectionsmagazine.com/article/ai...,2020-01-07,en,AI and Machine Learning | Connections Magazine,\n\nAI and Machine Learning | Connections Maga...,AI and Machine Magazine Connections Magazine...,Connections Magazine,AI and Machine Learning,...,['select monthjanuary 2020 11december 2019 1no...,select monthjanuary 2020 11december 2019 1nove...,2,[[2.3212399e-04 1.6879650e-04 9.9959904e-01]],2,"[('AI', 'PRODUCT'), ('Machine Magazine Connec...","['Machine Magazine Connections', 'Home Vendor...",['AI'],"['ServiceUs About', 'Peter DeHaan', 'Peter DeH...",[]
6,6.0,6,https://www.devdiscourse.com/article/business/...,2020-01-23,en,Air New Zealand trialing in computer vision AI...,\n\nAir New Zealand trialing in computer visio...,Air trialing in computer vision AI to improv...,NaN,Air New Zealand trialing in computer vision AI...,...,['caribbeanhowsresechlive diourseblogopionsubt...,caribbeanhowsresechlive diourseblogopionsubt p...,2,[[1.9538931e-04 3.4055416e-03 9.9639904e-01]],2,"[('Air', 'ORG'), ('AI', 'PRODUCT'), ('AsiaEast...","['Air', 'AsiaEast', 'Central AsiaCentral Afric...","['AI', 'AI', 'Apron AI', 'Apron AI', 'AI', 'AI...","['Willis', 'Willis', 'Jennifer Sepull', 'Assai...",['Democrats']
9,9.0,9,https://www.globenewswire.com/news-release/202...,2020-01-16,en,The global artificial intelligence (AI) in BFS...,\nThe global artificial intelligence (AI) in B...,The global artificial intelligence AI in BFSI...,NaN,The global artificial intelligence (AI) in BFS...,...,['intel corporationinternational business mach...,intel corporationinternational business machin...,6,[[2.2198893e-04 1.8831962e-04 9.9958974e-01]],2,"[('AI', 'PRODUCT'), ('17,765.2 million', 'CARD...","['GR', 'GR', 'Services', 'Solution Chatbots, F...","['AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI', 'AI...","['.Ipsoft .Zest FinanceRead', 'Clare']",['FranaisUs']
10,10.0,10,https://www.healthcareitnews.com/news/google-a...,2020-01-08,en,Google AI platform aids oncologists in breast ...,\nGoogle AI platform aids oncologists in breas...,Google AI platform aids oncologists in breast...,Healthcare IT News,Google AI platform aids oncologists in breast ...,...,['toggle navigation main menutopics video anal...,toggle navigation main menutopics video analyt...,1,[[1.8943226e-04 7.0772506e-04 9.9910283e-01]],2,"[('Google AI', 'ORG'), ('Google', 'ORG'), ('He...","['Google AI', 'Google', 'Healthcare ITstraight...","['DeepMind', 'AI', 'AI', 'AI', 'AI', 'AI']","['Nathan EddyJanuary 07', 'Xray', 'Shravya She...",[]


In [9]:
import ast
df_filt_pos['cleaned_text_tokens']=df_filt_pos['cleaned_text_tokens'].apply(ast.literal_eval)
df_filt_pos['cleaned_tokens_string'] = df_filt_pos['cleaned_text_tokens'].apply(' '.join)


In [10]:
df_filt_pos['cleaned_tokens_string'].head(2)

1    artificial intelligence researchers develop ai...
5    ai machine magazine connections magazineprimar...
Name: cleaned_tokens_string, dtype: object

In [11]:
import bertopic
from bertopic import BERTopic

2023-05-23 06:00:07.175969: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-23 06:00:13.978902: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-05-23 06:00:13.980341: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [ ]:
%%time

mod_BERT = BERTopic(calculate_probabilities=True)
topics, probabilities = mod_BERT.fit_transform(df_filt_pos.cleaned_tokens_string.tolist())

In [ ]:
print("Number of topics:", mod_BERT.get_topic_freq().shape[0])

In [ ]:
# Check the top words for each topic

for i, (k, t) in enumerate(mod_BERT.get_topics().items()):
    topic_words = [x[0] for x in t]
    print(f"Topic {k}: {', '.join(topic_words)}")

In [ ]:
mod_BERT.visualize_topics()

In [ ]:
mod_BERT.visualize_distribution(probabilities[0])